# Modular

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import time
from nltk.corpus import wordnet
import re
import enchant

# Read data

In [2]:
filename = '/Users/Lyf/OneDrive/study/WISC/2017_spring/Stat_628/hw2/data/train_data.csv'
reviews = pd.read_csv(filename)

# Test number

In [144]:
N = 100000

In [145]:
yelp = reviews[0:N].copy()
yelp_ori = yelp.copy

# 1 加上长度变量

In [146]:
yelp.loc[:, 'text length'] = yelp['text'].apply(len)

In [147]:
d = enchant.Dict("en_US")

# 2 改缩写

In [148]:
shortcut_dir = {'n\'t': ' not', '\'s': ' is', '\'ve': ' have', '\'d': ' would', '\'m': ' am', 'Ive': 'I have'}

In [149]:
t1 = time.time()
def shortcut_remove(review):
    for i in shortcut_dir:
        review = re.sub(i, shortcut_dir[i], review)
    return review
yelp2 = yelp.copy()
yelp2['text'] = yelp2['text'].apply(shortcut_remove)
time.time() - t1

0.998816728591919

# 3 Delete unEnglish

In [150]:
def check_english(word):
    return(d.check(word))
    
def unEnglish_remove(review):
    review = re.sub('[^a-zA-Z]',' ', review)
    words = pd.Series(review.split())
    index = words.apply(check_english)
    if(len(index)==0):
        return True
    else:
        return(sum(index)/len(index)>0.8)
t1 = time.time()
index = yelp2['text'].apply(unEnglish_remove)
time.time() - t1

263.9606430530548

In [151]:
yelp3 = yelp2[index].copy()

# Get positive and negative

In [80]:
reviews_total = ''
for i in range(N):
    reviews_total = reviews_total+" "+yelp.loc[i, 'text']

KeyboardInterrupt: 

In [ ]:
reviews_total
positive_words = ["nice","prime","great","awesome","yummy","friendly","delicious",
                  "love","spicy","good","favorite","well","clean","enjoy"]
negetive_words = ["suck","terrible","amazing","messy","worse","hit","miss"]

In [87]:
print(reviews['text'][9])

I saw this restaurant on a whim when I was looking for a good indian food place to try and decided that this was the one. We opted for delivery because why not.

I usually judge the place based on my all time favourite Butter chicken and when I got it for sure this place has it down! It surpassed the other ones I tried and for sure this is where am getting it from now on.

Aside from that dish, we also got the chicken tikka masala and the garlic naan bread. The masala was alright, I tasted a better one elsewhere so this one wasn't really my favourite from them. And the garlic naan was yummy. Really generous with the garlic for sure. 

And I also got a Mango lassi, I didnt like their version that much so I think I'll skip on this one next time.

Anyway, I have yet to try their other dishes in the future so for now am just giving them three stars. And also the other reason for the rating was because our food arrived late. Am glad I waited a little bit more because I was close to complain

# 4 去标点

In [154]:
def punctuation_remove(review):
    review = review.lower()
    review = re.sub("[^a-zA-Z]", " ", review)
    return review
t1 = time.time()
yelp4 = yelp3.copy()
yelp4['text'] = yelp4['text'].apply(punctuation_remove)
time.time() - t1

In [155]:
filename_chen = "../data/chen_first"+str(N)+".csv"
yelp4.to_csv(filename_chen, index = False)

# 5 去时态

In [158]:
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer 
stem = PorterStemmer()
def tense_remove(review):
    review = review.split()
    return " ".join([stem.stem(w) for w in review])
t1 = time.time()
yelp5 = yelp4.copy()
yelp5['text'] = yelp5['text'].apply(tense_remove)
time.time() - t1

209.00807809829712

# 6 Add not

In [160]:
def add_not(sentence):
    words = sentence.split().copy()
    not_index = 0
    for index, word in enumerate(words):
        if(not_index != 0):
            words[index] = 'not_' + words[index]
        if(re.match('.*[,.?!\'\"]$', word)):
            not_index = 0
        if(word == 'not'):
            not_index = 1
    #         print(index)
    sentence = " ".join(words)
    return(sentence)
t1 = time.time()
yelp6 = yelp5.copy()
yelp6['text'] = yelp6['text'].apply(add_not)
time.time() - t1

12.80996322631836

# 7 去停词

In [161]:
stops = set(stopwords.words("english"))

def noise_remove(review):
    review = review.split()
    useful_review = [w for w in review if not w in stops]  
    return " ".join([w for w in useful_review])
t1 = time.time()
yelp7 = yelp6.copy()
yelp7['text'] = yelp7['text'].apply(noise_remove)
time.time() - t1

2.492175817489624

In [162]:
filename_li = "../data/li_first"+str(N)+".csv"

In [165]:
yelp7.to_csv(filename_li, index = False)

In [84]:
filename

'first10000.csv'

In [87]:
yelp4['text'][3]

'long stori short bunch rude heartless ignor asshol run place compass peopl especi peopl feel belong suitabl shitti establish mother endur extrem humili situat bonjour brioch went saturday wait min cane nowher sit wait friend number peopl wait seat ask would get seat friend arriv would like coffe wait basic told friend tough shit seat wait ask well guest cancel last minut would like eat alon go serv even sit someth alon start tell get excus time could get tabl friend arriv ask manag friend arriv time explain manag wait cane given tabl peopl seat even wait other felt demean appreci servic would take busi somewher els instead manag make arrang eas situat told mother leav never life mother call cri phone humili embarrass felt absolut disgust'

In [164]:
filename

'first10000.csv'